## Deadline + Late Penalty

**Note :** It will take you quite some time to complete this project, therefore, we earnestly recommend that you start working as early as possible.


* Submission deadline for the Project is **20:59:59 on 24th Apr, 2020 (Sydney Time)**.
* **LATE PENALTY: Late Penalty: 10-% on day-1 and 20% on each subsequent day.**

## Instructions
1. This note book contains instructions for **COMP9318-Project**.

2. You are required to complete your implementation in a file `submission.py` provided along with this notebook.

3. You are not allowed to print out unnecessary stuff. We will not consider any output printed out on the screen. All results should be returned in appropriate data structures via corresponding functions.

4. You can submit your implementation for **Project** via following link: http://kg.cse.unsw.edu.au/submit/ (for students in China use https://unswkg.net/submit/).

5. For each question, we have provided you with detailed instructions along with question headings. In case of any problem, you can post your query @ Piazza.

6. You are allowed to add other functions and/or import modules (you may have for this project), but you are not allowed to define global variables. **Only functions are allowed** in `submission.py`. 

7. We only support the following modules/libraries, importing other modules will lead to errors. 
 * **Scipy 1.4.1**
 * **Numpy 1.81.6**
 * **Python 3.6**

8. We will provide immediate feedback on your submission **based on small sample testcases**. You can view the feedback using the online submission portal on the same day.

9. For **Final Evaluation** we will be using more different testcases, so your final scores **may vary** even you have passed the testcase. 

10. You are allowed to have a limited number of Feedback Attempts **(15 Attempts for each Team)**, we will use your **LAST** submission for Final Evaluation.

## Part1: PQ for $L_1$ Distance (45 Points)

In this question, you will implement the product quantization method with $L_1$ distance as the distance function. **Note** that due to the change of distance function, the PQ method introduced in the class no longer works. You need to work out how to adjust the method and make it work for $L_1$ distance. For example, the K-means clustering algorithm works for $L_2$ distance, you need to implement its $L_1$ variants (we denote it as K-means* in this project). You will also need to explain your adjustments in the report later.

Specifically, you are required to write a method `pq()` in the file `submission.py` that takes FOUR arguments as input:

1. **data** is an array with shape (N,M) and dtype='float32', where N is the number of vectors and M is the dimensionality.
2. **P** is the number of partitions/blocks the vector will be split into. Note that in the examples from the inverted multi index paper, P is set to 2. But in this project, you are required to implement a more general case where P can be any integer >= 2. You can assume that P is always divides M in this project. 
3. **init_centroids** is an array with shape (P,K,M/P) and dtype='float32', which corresponds to the initial centroids for P blocks. For each block, K M/P-dim vectors are used as the initial centroids. **Note** that in this project, K is fixed to be 256.
4. **max_iter** is the maximum number of iterations of the K-means* clustering algorithm. **Note** that in this project, the stopping condition of K-means* clustering is that the algorithm has run for ```max_iter``` iterations.

## Return Format (Part 1)

The `pq()` method returns a codebook and codes for the data vectors, where
* **codebooks** is an array with shape (P, K, M/P) and dtype='float32', which corresponds to the PQ codebooks for the inverted multi-index. E.g., there are P codebooks and each one has K M/P-dimensional codewords.
* **codes** is an array with shape (N, P) and dtype=='uint8', which corresponds to the codes for the data vectors. The dtype='uint8' is because K is fixed to be 256 thus the codes should integers between 0 and 255. 

### Manhattan distance

https://pdfs.semanticscholar.org/a630/316f9c98839098747007753a9bb6d05f752e.pdf  

### Product Quantization

https://hal.inria.fr/inria-00514462v1/document

In [2]:
from scipy.spatial import distance
import submission
import pickle
import time
import numpy as np
import queue as Q

In [3]:
def K_star(data,centroids,max_iter):  ## return 类型不对
    k = 1
    c_shape0 = centroids.shape[0]
    codes = np.empty((data.shape[0],1),dtype=np.uint8)
    codes[:] = np.NaN
    centroids = centroids.astype(np.float32)
    while k <= max_iter:     
        for i in range(data.shape[0]):
            d_manht = distance.cdist([data[i]],centroids,'cityblock')
            index = np.argmin(d_manht)
            codes[i] = index
        
        for j in range(c_shape0):
            lk = np.argwhere(codes == j)
            if lk.size > 0:
                cluster_list = [data[x[0]] for x in lk]
                centroids[j] = np.median(cluster_list,axis=0)
        k+=1
        
    for i in range(data.shape[0]):
            d_manht = distance.cdist([data[i]],centroids,'cityblock')
            index = np.argmin(d_manht)
            codes[i] = index
    return codes, centroids
    

In [4]:
def pq(data, P, init_centroids, max_iter): ##data is (768,128),init_centroids is (2,256,64)
    offset = init_centroids.shape[2]
    j = 0
    k = offset
    a = []
    b = []
    for i in range(P):
        data_x = data[:,j:k*(i+1)]  #shape 768,64
        codes, codebooks = K_star(data_x,init_centroids[i],max_iter)
        a.append(codebooks)
        b.append(codes)
        j = k*(i+1)
    codes = np.concatenate(b,axis=1)
    codebooks = np.array(a)
    return codebooks,codes 


In [5]:
with open('./example/CodeBooks_1', 'rb') as f:
    CodeBooks_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Data_File_1', 'rb') as f:
    Data_File_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Centroids_File_1', 'rb') as f:
    Centroids_File_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Codes_1', 'rb') as f:
    Codes_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
start = time.time()
codebooks, codes = pq(Data_File_1, P=2, init_centroids=Centroids_File_1, max_iter = 20)
end = time.time()
time_cost_1 = end - start
# print(time_cost_1)
# print(codebooks.dtype)
# print(codebooks)
# print("-------")
# print(CodeBooks_1.dtype)
# print(CodeBooks_1)
# print("-------")
# print(codes[:10])
# print("-0-----")
# print(Codes_1[:10])

np.savetxt('codes.txt', codes, delimiter=',')   # X is an array
np.savetxt('Codes_1.txt', Codes_1, delimiter=',')   # X is an array
#np.savetxt('codebooks.txt', codebooks, delimiter=',')   # X is an array
#np.savetxt('', CodeBooks_1, delimiter=',')   # X is an array
# ll = np.array([Data_File_1[:,:64][x] for x in range(3)])
# print(Data_File_1[:,:64])
# print("--------")
# print(ll)
# print("--------")
# print(np.median(ll,axis=0))
# print(np.median(ll[:,1]))
# print(codebooks)
# print(CodeBooks_1)
# print(codes)
# print("-------")
# print(Codes_1)

In [148]:
with open('./example/CodeBooks_2', 'rb') as f:
    CodeBooks_2 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Data_File_2', 'rb') as f:
    Data_File_2 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Centroids_File_2', 'rb') as f:
    Centroids_File_2 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Codes_2', 'rb') as f:
    Codes_2 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Candidates_2', 'rb') as f:
    Candidates_2 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
start = time.time()
codebooks2, codes2 = pq(Data_File_2, P=4, init_centroids=Centroids_File_2, max_iter = 20)
end = time.time()
time_cost_1 = end - start
print(time_cost_1)
# print(time_cost_1)
# np.savetxt('codes2.txt', codes2, delimiter=',')   # X is an array
# np.savetxt('Codes_2.txt', Codes_2, delimiter=',')   # X is an array
# print(codebooks2.dtype)
# print(codebooks2)
# print("-------")
# print(CodeBooks_2.dtype)
# print(CodeBooks_2)
# print("-------")
# print(codes2.dtype)
# print(codes2)
# print("-0-----")
# print(Codes_2.dtype)
# print(Codes_2)

4.855967044830322


In [6]:
# How to run your implementation for Part 1
with open('./toy_example/Data_File', 'rb') as f:
    Data_File = pickle.load(f, encoding = 'bytes') #shape 768,128
with open('./toy_example/Centroids_File', 'rb') as f:
    Centroids_File = pickle.load(f, encoding = 'bytes') #shape 2,256,64
# print("ds",Centroids_File)
# print("ds",Data_File)
start = time.time()
codebooks, codes = pq(Data_File, P=2, init_centroids=Centroids_File, max_iter = 20)
end = time.time()
time_cost_1 = end - start
# print(time_cost_1)
# #print(type(codes))
# #print(codebooks.shape)
# #print(codebooks)
# print(codes.dtype)
# print(codebooks.dtype)
# print(Centroids_File)
#a = Data_File[:,:64]
# for i in a:
#     print(np.argmin(distance.cdist([i],Centroids_File[0,:,:64],'cityblock')))

In [539]:
nn = np.empty((3,4))
nn[:] = np.NaN
nn[:] = 1
nn.astype(np.uint8)
np.argwhere(nn == 1)[0][0]

0

# Part2: Query using Inverted Multi-index with $L_1$ Distance (45 Points)

In this question, you will implement the query method using the idea of inverted multi-index with $L_1$ distance. Specifically, you are required to write a method `query()` in the file `submission.py` that takes arguments as input:

1. **queries** is an array with shape (Q, M) and dtype='float32', where Q is the number of query vectors and M is the dimensionality.
2. **codebooks** is an array with shape (P, K, M/P) and dtype='float32', which corresponds to the `codebooks` returned by `pq()` in part 1.
3. **codes** is an array with shape (N, P) and dtype=='uint8', which corresponds to the `codes` returned by `pq()` in part 1.
4. **T** is an integer which indicates the minimum number of returned candidates for each query. 

## Return Format (Part 2)

The `query()` method returns an array contains the candidates for each query. Specifically, it returns
* **candidates** is a list with Q elements, where the i-th element is a **set** that contains at least T integers, corresponds to the id of the candidates of the i-th query. For example, assume $T=10$, for some query we have already obtained $9$ candidate points. Since $9 < T$, the algorithm continues. Assume the next retrieved cell contains $3$ points, then the returned set will contain $12$ points in total.

## Implementation Hints

The implementation of `query()` should be efficiency. You should work out at least
1. How to efficiently extend Algorithm 3.1 in the paper to a general case with P > 2.
2. How to efficiently make use of `codes` returned by Part 1. For example, it may not be wise to enumerate all the possible combinations of codewords to build the inverted index. 

We will test the efficiency by setting a running time limit (more details later).

In [288]:
k = [0,0]
#np.argwhere(codes==k)
#np.squeeze(codes)
d = np.argwhere(codes[:,0]==k[0])#.flatten()
f = np.argwhere(codes[:,1]==k[1])#.flatten()
k = np.intersect1d(d,f)
a = set()
a.update(k)
a


{0, 1, 2}

In [37]:
def traverse(traverse_arr,p,matrix_index,ll,x):
    isTrue = True
    for g in range(x):
        cl = [i for i in tuple(ll)]
        if g != p:
            cl[g] = cl[g] - 1
            if matrix_index[g]==0 or tuple(cl) in traverse_arr:
                isTrue = True
            else:
                isTrue = False
                return isTrue
    return isTrue
    
    
    
    
#     for i in range(x):
#         if traverse_arr[i,y,matrix_index[i]] == 0:
#             isTrue = 0
#             return isTrue
#         else:
#             isTrue = 1
#     return isTrue

In [74]:
# start = time.time()
# codebooks, codes = submission.pq(Data_File, P=2, init_centroids=Centroids_File, max_iter = 20)
# end = time.time()
# time_cost_1 = end - start

with open('./example/CodeBooks_1', 'rb') as f:
    CodeBooks_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Data_File_1', 'rb') as f:
    Data_File_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Centroids_File_1', 'rb') as f:
    Centroids_File_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Codes_1', 'rb') as f:
    Codes_1 = pickle.load(f, encoding = 'bytes') #shape 2,256,64
start = time.time()
codebooks, codes = pq(Data_File_1, P=2, init_centroids=Centroids_File_1, max_iter = 20)
end = time.time()
time_cost_1 = end - start



In [161]:
def multi_seq(dist_matrix,dist_index_matrix,d_codes,T):
    x = dist_matrix.shape[0]  
    y = dist_matrix.shape[1]  
    z = dist_matrix.shape[2]  
    
    points_list = []
    
    for i in range(y):
        traverse_arr = tuple()
        queue = Q.PriorityQueue()
        points_set = set()
        queue.put((sum(dist_matrix[:,i,0]),(0,)*x)) 
        while len(points_set) < T:
            centroid = queue.get()
            matrix_index = centroid[1]
            traverse_arr += (matrix_index,)
            tt = tuple(dist_index_matrix[j,i,matrix_index[j]] for j in range(x))

            if tt in d_codes:
                points_set.update(d_codes[tt])

            for p in range(x):
                if matrix_index[p] < z: 
                    ll = list(matrix_index)
                    ll[p] = ll[p] + 1
                    cl = tuple(c for c in ll)
                    if traverse(traverse_arr,p,matrix_index,ll,x):
                        queue.put((sum([dist_matrix[g,i,cl[g]] for g in range(x)]),cl))
                
        points_list.append(points_set)
    return points_list                   
                        


In [162]:
def multi_seq2(dist_matrix,dist_index_matrix,d_codes,T):
    x = dist_matrix.shape[0]  
    z = dist_matrix.shape[1]  
    #z = dist_matrix.shape[2]  
    
    #for i in range(y):
    traverse_arr = tuple()
    queue = Q.PriorityQueue()
    points_set = set()
    queue.put((sum(dist_matrix[:,0]),(0,)*x)) 
    while len(points_set) < T:
        centroid = queue.get()
        matrix_index = centroid[1]
        traverse_arr += (matrix_index,)
        tt = tuple(dist_index_matrix[j,matrix_index[j]] for j in range(x))

        if tt in d_codes:
            points_set.update(d_codes[tt])

        for p in range(x):
            if matrix_index[p] < z: 
                ll = list(matrix_index)
                ll[p] = ll[p] + 1
                cl = tuple(c for c in ll)
                if traverse(traverse_arr,p,matrix_index,ll,x):
                    queue.put((sum([dist_matrix[g,cl[g]] for g in range(x)]),cl))

    return points_set            

In [178]:
def query(queries,codebooks,codes,T): #queries with shape 10,2,64 (10 queries 2 partition 64 dimension)
    j = 0 
    dist_matrix = []
    dist_index_matrix = []
    points_list = []
    #print("codes",codebooks[1].shape[1])
    d_codes = {}
    for i in range(len(codes)):
        tp = tuple(codes[i])
        if tp in d_codes:
            d_codes[tp].append(i)
        else:
            d_codes[tp] = [i]
    #print(d_codes)
    for e in codebooks:
        i = j
        j += e.shape[1]
        sub_query = queries[:,i:j]
        d = distance.cdist(sub_query,e,'cityblock')
        sort_d = np.sort(d)
        argsort_d = np.argsort(d)
        dist_matrix.append(sort_d)
        dist_index_matrix.append(argsort_d)
    dist_matrix = np.array(dist_matrix)
    dist_index_matrix = np.array(dist_index_matrix)
#     for k in range(queries.shape[0]):
#         points_set = multi_seq2(dist_matrix[:,k,:],dist_index_matrix[:,k,:],d_codes,T)
#         points_list.append(points_set)
#     return points_list
#     #print(dist_matrix)
    points_list = multi_seq(dist_matrix,dist_index_matrix,d_codes,T)
    return points_list
#     print(points_list)
#     return points_list
#     print("my set",points_list)
    #print(dist_matrix.shape)
    #print(dist_matrix)
        #print(sub_query[:,None] - e[:,:,None])
        #print(e)
        #print(sub_query,sub_query.shape)
    #print(queries.shape,codebooks.shape,codes.shape)
    
    

In [179]:
# How to run your implementation for Part 2
with open('./toy_example/Query_File', 'rb') as f:
    Query_File = pickle.load(f, encoding = 'bytes')

## query 1
with open('./example/Query_File_1', 'rb') as f:
    Query_File = pickle.load(f, encoding = 'bytes')
with open('./example/CodeBooks_1', 'rb') as f:
    codebooks = pickle.load(f, encoding = 'bytes') #shape 2,256,64
with open('./example/Codes_1', 'rb') as f:
    codes = pickle.load(f, encoding = 'bytes') #shape 2,256,64
############

## query 2
# with open('./example/Query_File_2', 'rb') as f:
#     Query_File = pickle.load(f, encoding = 'bytes')
# with open('./example/CodeBooks_2', 'rb') as f:
#     codebooks = pickle.load(f, encoding = 'bytes') #shape 2,256,64
# with open('./example/Codes_2', 'rb') as f:
#     codes = pickle.load(f, encoding = 'bytes') #shape 2,256,64
#############

queries = Query_File#pickle.load(Query_File, encoding = 'bytes')
queries = queries.reshape(10,128)
start = time.time()
candidates = query(queries, codebooks, codes, T=100)
end = time.time()
time_cost_2 = end - start
print(time_cost_2)
print(candidates)
# k = dict((tuple(codes[i]),i) for i in range(len(codes)))
# k

# d_codes = {}
# for i in range(len(codes)):
#     tp = tuple(codes[i])
#     if tp in d_codes:
#         d_codes[tp].update([i])
#     else:
#         #d_codes[tp] = set()
#         d_codes[tp].update(set([i]))
# d_codes

#len(k)
#tuple(codes[0])
#print(queries)
#print(queries.reshape(10,128))


# a = codebooks[0]
# #b = queries[:2,]
# c = queries[:2,0]
# # #print(Data_File.shape)
# # # output for part 2.
# # #print(candidates)
# # print("codebook",a)
# # print("query",c)
# # print(a.shape,c.shape)
# d = abs(c[:,None] - a).sum(-1)
# i = np.argsort(d)
# j = np.sort(d)
# print(i[1])
# print(j[1])
# index_value = np.dstack((i,j))
# #print(index_value.shape)
# index_value
# print(d[0].shape)
# print(d.shape)



4.248894929885864
[{8, 13, 14, 18, 19, 26, 31, 34, 35, 37, 44, 47, 50, 52, 54, 73, 76, 79, 98, 108, 111, 129, 139, 143, 148, 151, 162, 165, 166, 176, 182, 184, 188, 190, 197, 201, 216, 235, 236, 237, 241, 243, 246, 247, 250, 254, 258, 262, 263, 264, 269, 274, 276, 279, 281, 287, 299, 301, 319, 325, 347, 350, 352, 355, 357, 362, 366, 368, 369, 371, 372, 373, 378, 381, 398, 401, 404, 405, 414, 417, 423, 425, 429, 431, 433, 434, 445, 446, 448, 452, 457, 458, 465, 469, 471, 472, 474, 475, 485, 495}, {13, 15, 17, 19, 22, 25, 34, 35, 42, 50, 51, 52, 61, 69, 76, 84, 86, 90, 97, 101, 121, 128, 129, 134, 138, 143, 145, 147, 150, 156, 165, 166, 174, 184, 190, 193, 195, 199, 203, 212, 214, 217, 224, 230, 235, 241, 243, 245, 248, 251, 260, 269, 271, 274, 279, 281, 286, 291, 298, 299, 311, 312, 315, 317, 320, 346, 349, 350, 352, 355, 357, 366, 369, 375, 377, 383, 386, 392, 394, 404, 410, 411, 413, 414, 416, 423, 424, 425, 433, 448, 456, 457, 461, 462, 474, 480, 482, 488, 495, 499}, {1, 6, 16, 18, 2

In [173]:
q = Q.PriorityQueue()
q.put((10,[1,1]))
q.put((2,[1,2]))
q.put((1,[2,3]))
q.get()
q.get()
a = tuple()
l = (3,4)
a = a + (l,)
if (3,4) in a:
    print("yes")
ll = [3,4,5,6]
ll = np.array(ll)
a = tuple(ll[i] for i in range(len(ll)))
a = (0,) * 4
k = list(a)
k[0] = k[0]+1
print(a,k)

yes
(0, 0, 0, 0) [1, 0, 0, 0]


In [76]:
candidates

[{13, 34, 197, 243, 254, 274, 281, 404, 433, 458},
 {15, 184, 235, 241, 317, 377, 410, 414, 425, 482},
 {42, 69, 88, 218, 228, 229, 279, 335, 382, 458},
 {34, 37, 53, 162, 371, 397, 423, 433, 438, 445},
 {80, 164, 240, 241, 247, 264, 275, 329, 355, 423},
 {16, 65, 187, 190, 230, 355, 357, 396, 439, 446},
 {15, 67, 129, 164, 236, 319, 331, 381, 394, 398},
 {19, 35, 85, 188, 218, 305, 355, 365, 406, 413},
 {177, 184, 229, 347, 351, 355, 389, 411, 446, 447},
 {66, 82, 90, 98, 148, 155, 250, 312, 340, 416}]

In [189]:
len(codes)
{163, 264, 367, 176, 431, 466, 81, 404, 247, 88}

500

In [131]:
with open('./example/Candidates_1', 'rb') as f:
    Candidate_File = pickle.load(f, encoding = 'bytes')
with open('./example/Candidates_2', 'rb') as f:
    Candidate_File_2 = pickle.load(f, encoding = 'bytes')
# for i in range(len(Candidate_File)):
# #     if (Candidate_File[i]==candidates[i]):
# #         print(i)
# #     else:
# #         print("wrong line",i)
#     print(i,"---",Candidate_File[i]-candidates[i])
Candidate_File_2

[{3,
  28,
  38,
  57,
  72,
  82,
  83,
  89,
  103,
  130,
  146,
  150,
  160,
  163,
  167,
  182,
  187,
  197,
  202,
  209,
  212,
  214,
  221,
  229,
  231,
  233,
  236,
  247,
  259,
  277,
  285,
  316,
  318,
  321,
  323,
  352,
  367,
  369,
  371,
  375,
  382,
  389,
  392,
  393,
  413,
  414,
  419,
  422,
  423,
  489,
  504,
  512,
  516,
  536,
  538,
  543,
  544,
  545,
  557,
  588,
  592,
  601,
  615,
  626,
  628,
  634,
  642,
  643,
  650,
  659,
  660,
  668,
  673,
  675,
  690,
  706,
  726,
  752,
  756,
  761,
  762,
  763,
  767,
  770,
  779,
  784,
  802,
  807,
  828,
  834,
  873,
  882,
  900,
  910,
  936,
  950,
  962,
  972,
  989,
  996},
 {2,
  4,
  13,
  16,
  26,
  29,
  32,
  49,
  51,
  57,
  60,
  65,
  78,
  83,
  94,
  103,
  105,
  108,
  115,
  130,
  134,
  172,
  178,
  193,
  206,
  209,
  244,
  250,
  255,
  272,
  274,
  279,
  291,
  309,
  310,
  311,
  312,
  316,
  322,
  325,
  330,
  345,
  352,
  356,
  359,
  370,
  4

In [49]:
queries.shape

(2, 256, 64)

# Evaluation

Your implementation will be tested using 3 testcases (**30 points each, and another 10 points from the report**), your result will be compared with the result of the correct implementation. Part 1 and part 2 will be test **seperately** (e.g., you may still get 45 points from part 2 even if you do not attempt part 1), and you will get full mark for each part if the output of your implementation matches the expected output and 0 mark otherwise. 

**Note:** One of these 3 testcases is the same as the one used in the **submssion system**.

### The Inverted Multi-Index
http://sites.skoltech.ru/app/data/uploads/sites/25/2014/12/TPAMI14.pdf

## How to run your implementation (Example)

In [356]:
import submission
import pickle
import time

# How to run your implementation for Part 1
with open('./toy_example/Data_File', 'rb') as f:
    Data_File = pickle.load(f, encoding = 'bytes')
with open('./toy_example/Centroids_File', 'rb') as f:
    Centroids_File = pickle.load(f, encoding = 'bytes')
start = time.time()
codebooks, codes = submission.pq(Data_File, P=2, init_centroids=Centroids_File, max_iter = 20)
end = time.time()
time_cost_1 = end - start


# How to run your implementation for Part 2
with open('./toy_example/Query_File', 'rb') as f:
    Query_File = pickle.load(f, encoding = 'bytes')
queries = Query_File
start = time.time()
candidates = submission.query(queries, codebooks, codes, T=10)
end = time.time()
time_cost_2 = end - start

# output for part 2.
print(candidates)

None


In [133]:
a = set()
b = [1,2,3]
a.update(b)
a

{1, 2, 3}

## Running Time Limits

As shown in the above snippet, we will be recording the running time of both part 1 and part 2. Your implementation is expected to finish with Allowed time Limit. If your code takes longer than Allowed Time Limit, your program will be terminated and you will recieve 0 mark.

For example, on CSE machine, e.g., **wagner**, your code is supposed to finish with in 3 seconds (for part 1) and 1 second (for part 2) for the toy example illustrated above.

## Project Submission and Feedback

For project submission, you are required to submit the following files:

1. Your implementation in a python file `submission.py`.

2. A report `Project.pdf` (**10 points**). You need to write a concise and simple report illustrating
    - Implementation details of part 1, especially what changes you made to accomodate $L_1$ distance.
    - Implementation details of part 2, including the details on how you extended the algorithm 3.1 to a more general case with P>2, and how you efficiently retrieve the candidates. 


**Note:** Every team will be entitled to **15 Feedback Attempts** (use them wisely), we will use the last submission for final evaluation.